In [1]:
from __future__ import annotations

import json
from pathlib import Path

import pandas as pd

ROOT = Path(".").resolve().parent if Path(".").resolve().name == "notebooks" else Path(".").resolve()

ART_BEH = ROOT / "artifacts" / "behavior"
ART_CHURN = ROOT / "artifacts" / "churn"
ART_NLP = ROOT / "artifacts" / "nlp"

print("ROOT:", ROOT)
print("behavior:", ART_BEH)
print("churn:", ART_CHURN)
print("nlp:", ART_NLP)

# churn
churn_metrics = json.loads((ART_CHURN / "metrics.json").read_text(encoding="utf-8")) if (ART_CHURN / "metrics.json").exists() else {}
# behavior
cluster_profiles = pd.read_csv(ART_BEH / "cluster_profiles.csv") if (ART_BEH / "cluster_profiles.csv").exists() else pd.DataFrame()
churn_by_cluster = pd.read_csv(ART_BEH / "churn_by_cluster.csv") if (ART_BEH / "churn_by_cluster.csv").exists() else pd.DataFrame()
# nlp
nlp_metrics = json.loads((ART_NLP / "metrics.json").read_text(encoding="utf-8")) if (ART_NLP / "metrics.json").exists() else {}
subtopics = json.loads((ART_NLP / "subtopics.json").read_text(encoding="utf-8")) if (ART_NLP / "subtopics.json").exists() else {}

(churn_metrics, list(nlp_metrics.keys())[:5], subtopics.get("target_topic"))


ROOT: C:\Users\User\Desktop\dev\hakaton_mipt
behavior: C:\Users\User\Desktop\dev\hakaton_mipt\artifacts\behavior
churn: C:\Users\User\Desktop\dev\hakaton_mipt\artifacts\churn
nlp: C:\Users\User\Desktop\dev\hakaton_mipt\artifacts\nlp


({'rows': 135867,
  'churn_rate': 0.6781190428875297,
  'roc_auc': 0.7068427290569274,
  'pr_auc': 0.8264758746170506},
 ['rows',
  'topics',
  'labels_order',
  'classification_report',
  'confusion_matrix'],
 'Портал')

In [2]:
display(pd.DataFrame([{
    "rows": churn_metrics.get("rows"),
    "churn_rate": churn_metrics.get("churn_rate"),
    "roc_auc": churn_metrics.get("roc_auc"),
    "pr_auc": churn_metrics.get("pr_auc"),
}]))


,rows,churn_rate,roc_auc,pr_auc
0,135867,0.678119,0.706843,0.826476


In [3]:
print("Cluster profiles (means):")
display(cluster_profiles)

print("Churn by cluster:")
display(churn_by_cluster.sort_values("churn_rate", ascending=False) if not churn_by_cluster.empty else churn_by_cluster)


Cluster profiles (means):


,cluster,device_id,events_cnt,active_days,sessions_cnt,screens_cnt,features_cnt,pay_events,ticket_events,meter_events
0,0,97711.882167,7.169778,2.542513,3.122418,1.689066,2.628028,0.0,0.327867,0.0
1,1,61564.871080,50.724609,8.245640,12.452683,4.019693,9.515312,0.0,7.456092,0.0
2,2,24943.319132,385.539389,32.633441,85.782958,4.993569,15.692122,0.0,84.621383,0.0


Churn by cluster:


,cluster,users,churn_rate
0,0,142863,0.721810
1,1,35495,0.361741
2,2,1244,0.063505


In [4]:
def describe_cluster(row: pd.Series) -> str:
    parts = []
    if row.get("events_cnt", 0) > cluster_profiles["events_cnt"].mean():
        parts.append("высокая активность")
    else:
        parts.append("низкая активность")

    if row.get("pay_events", 0) > cluster_profiles["pay_events"].mean():
        parts.append("оплата")
    if row.get("ticket_events", 0) > cluster_profiles["ticket_events"].mean():
        parts.append("заявки")
    if row.get("meter_events", 0) > cluster_profiles["meter_events"].mean():
        parts.append("показания")

    if row.get("features_cnt", 0) > cluster_profiles["features_cnt"].mean():
        parts.append("широкое использование функций")
    else:
        parts.append("узкий сценарий")

    return ", ".join(parts)


if cluster_profiles.empty:
    print("cluster_profiles is empty")
else:
    desc = cluster_profiles.copy()
    desc["description_draft"] = desc.apply(describe_cluster, axis=1)

    if not churn_by_cluster.empty:
        desc = desc.merge(churn_by_cluster[["cluster", "churn_rate", "users"]], on="cluster", how="left")

    display(desc.sort_values("churn_rate", ascending=False) if "churn_rate" in desc.columns else desc)


,cluster,device_id,events_cnt,active_days,sessions_cnt,screens_cnt,features_cnt,pay_events,ticket_events,meter_events,description_draft,churn_rate,users
0,0,97711.882167,7.169778,2.542513,3.122418,1.689066,2.628028,0.0,0.327867,0.0,"низкая активность, узкий сценарий",0.721810,142863
1,1,61564.871080,50.724609,8.245640,12.452683,4.019693,9.515312,0.0,7.456092,0.0,"низкая активность, широкое использование функций",0.361741,35495
2,2,24943.319132,385.539389,32.633441,85.782958,4.993569,15.692122,0.0,84.621383,0.0,"высокая активность, заявки, широкое использова...",0.063505,1244


In [5]:
print("NLP topic distribution (coarse):")
topics = nlp_metrics.get("topics", {})
display(pd.DataFrame([{"topic": k, "count": v} for k, v in topics.items()]).sort_values("count", ascending=False))

print("\nTop subtopics inside target topic:", subtopics.get("target_topic"))
topics_list = subtopics.get("topics", [])
display(pd.DataFrame([{
    "topic_id": t.get("topic_id"),
    "terms": ", ".join(t.get("terms", [])[:10]),
    "example_1": (t.get("examples") or [""])[0],
} for t in topics_list]))


NLP topic distribution (coarse):


,topic,count
0,Портал,2388
1,Приложение/Умные решения,110
2,Другое,35
3,ОСС,12
4,Без темы,5
5,Оплата/ЕПД,3



Top subtopics inside target topic: Портал


,topic_id,terms,example_1
0,0,"как, на, осс, здравствуйте, что, для, подскажи...",Как повести ОСС?
1,1,"не могу, могу, не, могу проголосовать, проголо...",Не могу проголосовать
2,2,"опрос, шлагбаума, по, опрос по, не отображаетс...","Добрый день. Проверьте, пжл. В доме идëт опрос..."
3,3,"код, плательщика, код плательщика, добавить ко...",Не меняетсяс код плательщика
4,4,"день, добрый день, добрый, день не, день почем...","Добрый день, почему?"
5,5,"заявку, подать, подать заявку, отопления, заяв...",Невозможно подать заявку на отсутствие отопления
6,6,"34, заявки, при, 34 34, мои, 34 мои, не, ошибк...",Здравствуйте! Провожу сейчас ОСС на вашей площ...
7,7,"адресу, по, по адресу, ул, дом, прошу, москва,...",Добрый день! Сосед К является собственником кв...
8,8,"за, епд, баллы, почему, не, оплату, за оплату,...",Добрый день почему не начислены баллы за оплат...
9,9,"адрес, ошибку, выдает, выдает ошибку, ру, мос,...",На мос.ру зарегистрирован адрес. На не могу д...


In [6]:
recs = []

# churn+clusters
if not churn_by_cluster.empty:
    worst = churn_by_cluster.sort_values("churn_rate", ascending=False).iloc[0]
    recs.append(f"Кластер {int(worst['cluster'])} имеет максимальный churn_rate={worst['churn_rate']:.3f}: нужен онбординг/подсказки и триггерные уведомления для возврата.")

# NLP
target_topic = subtopics.get("target_topic")
if target_topic:
    recs.append(f"По обращениям доминирует тема '{target_topic}': приоритизировать продуктовые/поддержочные улучшения внутри этой зоны, используя под-темы из NMF.")

# adoption/retention general
recs.append("Рост retention D7 — ключевая цель: усилить сценарии, которые коррелируют с низким churn (оплата/заявки/показания), через персональные подсказки и упрощение пути.")
recs.append("Запустить A/B: 1) напоминания о показаниях, 2) упрощение оплаты, 3) быстрые шаблоны заявок. Метрики: D7, доля успешных действий, churn_14d.")

print("\nRECOMMENDATIONS (paste to slides):")
for i, r in enumerate(recs, 1):
    print(f"{i}) {r}")



RECOMMENDATIONS (paste to slides):
1) Кластер 0 имеет максимальный churn_rate=0.722: нужен онбординг/подсказки и триггерные уведомления для возврата.
2) По обращениям доминирует тема 'Портал': приоритизировать продуктовые/поддержочные улучшения внутри этой зоны, используя под-темы из NMF.
3) Рост retention D7 — ключевая цель: усилить сценарии, которые коррелируют с низким churn (оплата/заявки/показания), через персональные подсказки и упрощение пути.
4) Запустить A/B: 1) напоминания о показаниях, 2) упрощение оплаты, 3) быстрые шаблоны заявок. Метрики: D7, доля успешных действий, churn_14d.
